# DI 725: Transformers and Attention-Based Deep Networks

## Assignment 2 : Object Detection

The purpose of this notebook is to guide you through the usage of **auair_yolos.py.**

### Author:
* Ebru Kültür Başaran

## Requirements
Install requirements for your environment, comment out for later uses.

Dependencies:
- Python >=3.8
- pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu117
- pip install transformers[torch] albumentations opencv-python pycocotools torchmetrics wandb pillow

In [1]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 961.5/961.5 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 126.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 54.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninsta

In [3]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: trial to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


## 1. Convert annotations to COCO format
We convert to COCO format because the HuggingFace AutoImageProcessor and Trainer workflows expect COCO-style annotation JSON. Converting once at the start avoids needing custom parsing logic later and lets us leverage standardized COCO utilities such as pycocotools for data loading, augmentation, and evaluation.

In [ ]:
!python "/content/drive/MyDrive/Assignment 2/auair_yolos.py" convert \
    --ann "/content/drive/MyDrive/Assignment 2/data/AU-AIR/auair_coco.json" \
    --img-root "/content/drive/MyDrive/Assignment 2/data/AU-AIR/images" \
    --out "/content/drive/MyDrive/Assignment 2/yolos-auair"

## 2. Train YOLOS-Tiny Model
To train the model, we define the image root path, COCO converted annotations file, image size, number of epochs, number of batches, number of workers for optimum GPU usage and otput directory to save findings. For a sufficient training time, the epoch count is selected as 5.

In [26]:
!python "/content/drive/MyDrive/Assignment 2/auair_yolos.py" train \
  --img-root   "/content/drive/MyDrive/Assignment 2/data/AU-AIR/images" \
  --ann        "/content/drive/MyDrive/Assignment 2/data/AU-AIR/auair_coco.json" \
  --img-size 384 \
  --epochs   10 \
  --batch    8 \
  --workers  8 \
  --outdir   "/content/drive/MyDrive/Assignment 2/yolos-auair"

2025-04-21 23:42:15.376871: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-21 23:42:15.394950: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745278935.417335   34528 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745278935.424036   34528 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-21 23:42:15.446448: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

## 3. Evaluate the Model
We finally evaluate our models performance on test set. The fine‑tuned model is loaded from --model-dir, then the validation dataset is built on the test samples, and the Hugging Face Trainer.evaluate method to compute per‑class AP@0.5 and overall mAP s used. Finally, the resulting metrics dictionary is obtained.

In [28]:
!python "/content/drive/MyDrive/Assignment 2/auair_yolos_evaluation.py" \
  --img-root   "/content/drive/MyDrive/Assignment 2/data/AU-AIR/images" \
  --ann        "/content/drive/MyDrive/Assignment 2/yolos-auair/val.json" \
  --img-size   384 \
  --batch      8 \
  --workers    8 \
  --checkpoint "/content/drive/MyDrive/Assignment 2/yolos-auair/"

2025-04-22 00:37:28.494498: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-22 00:37:28.512173: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745282248.533901   50016 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745282248.540482   50016 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-22 00:37:28.562299: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr